In [ ]:
%pip install --upgrade -r requirements.txt

Create the Pay-i and OpenAI clients

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from payi import Payi

load_dotenv()

# Read the API KEYs from the environment, replace the default values (the second argument) with your own keys if needed
openai_key = os.getenv("OPENAI_API_KEY", "YOUR_OPENAPI_API_KEY")
payi_api_key= os.getenv("PAYI_API_KEY", "YOUR_PAYI_API_KEY")

payi = Payi(
    api_key=payi_api_key,
)

client = OpenAI(
    api_key=openai_key,
)

Create a use case definition and limit.  Both will be used when ingesting data into Pay-i

In [ ]:
use_case_name = 'openai_ingest_use_case_test'
payi.use_cases.definitions.create(name=use_case_name, description='This is a test use case')
 
result = payi.limits.create(max=10_000, limit_name='openai_ingest_limit')
limit_id = result.limit.limit_id

print(f"Created limit with id {limit_id}")

Instrument the OpenAI client so that the use case is associated with each inference call

In [ ]:
from payi.lib.instrument import payi_instrument

# Not passing payi as a parameter requires the env var "PAYI_API_KEY" is set, otherwise pass payi=payi
# When use_case_name is provided and use_id is not, a use instance id will be automatically created and the same id will be applied to all OpenAI calls
payi_instrument(config={"use_case_name": use_case_name})


Helper method to call the OpenAI client and pass additional information to Pay-i 

In [ ]:
from payi.lib.helpers import create_headers
from payi.lib.instrument import track_context

def call_openai(client, limit_id, user_id):
    
    with track_context(request_tags=['x', 'y'], user_id=user_id, limit_ids=[limit_id]):
        response = client.chat.completions.create(
            model="gpt-3.5-turbo", 
            messages=[{"role": "user", "content": "tell me a very short story"}],
        )
    
        completion = response.choices[0].message.content
        print(completion)

        # Additional calls to OpenAI can be made here and will be ingested.
        # If another method is called, it's calls to OpenAI will also be ingested.
    ...


Call OpenAI and the results are automatically ingested by the Pay-i instrumentation

In [ ]:
call_openai(client, limit_id, 'test_user_id')

See that the limit input, output, and total base cost has been updated with the ingest data

In [ ]:
response = payi.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.total.base}")

Reset a limit back to zero tracked cost

In [ ]:
print("\nState prior to reset:")
response = payi.limits.reset(limit_id=limit_id)
print(f"Limit Name: {response.limit_history.limit_name}")
print(f"Limit ID: {response.limit_history.limit_id}")
print(f"Limit Reset Timestamp: {response.limit_history.limit_reset_timestamp}")
print(f"Limit Tags: {response.limit_history.limit_tags}")
print(f"Limit Input Base Cost: {response.limit_history.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit_history.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.total.base}")